In [11]:
import pandas as pd
import pmdarima as pm
from datetime import datetime
from pmdarima import model_selection

# Lee el archivo como un DataFrame
#df = pd.read_csv('/Users/micaelabassan/Desktop/fundamentos/Labo 3/sell-in.txt', delimiter='\t',  decimal='.')  # Cambia el delimitador si es necesario
df  = pd.read_csv('data/sell-in.csv', delimiter='\t')
df_pid_validos  = pd.read_csv('data/productos_a_predecir.txt')
# Muestra las primeras filas del DataFrame
print(df.head())
print(df_pid_validos.head())

   periodo  customer_id  product_id  plan_precios_cuidados  cust_request_qty  \
0   201701        10234       20524                      0                 2   
1   201701        10032       20524                      0                 1   
2   201701        10217       20524                      0                 1   
3   201701        10125       20524                      0                 1   
4   201701        10012       20524                      0                11   

   cust_request_tn       tn  
0          0.05300  0.05300  
1          0.13628  0.13628  
2          0.03028  0.03028  
3          0.02271  0.02271  
4          1.54452  1.54452  
   product_id
0       20001
1       20002
2       20003
3       20004
4       20005


In [12]:
product_ids_validos = df_pid_validos['product_id'].tolist()

In [13]:
df.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.05300,0.05300
1,201701,10032,20524,0,1,0.13628,0.13628
2,201701,10217,20524,0,1,0.03028,0.03028
3,201701,10125,20524,0,1,0.02271,0.02271
4,201701,10012,20524,0,11,1.54452,1.54452


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2945818 entries, 0 to 2945817
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   periodo                int64  
 1   customer_id            int64  
 2   product_id             int64  
 3   plan_precios_cuidados  int64  
 4   cust_request_qty       int64  
 5   cust_request_tn        float64
 6   tn                     float64
dtypes: float64(2), int64(5)
memory usage: 157.3 MB


In [15]:
# Filtrar el DataFrame original para quedarse solo con los product_id válidos
data = df[df['product_id'].isin(product_ids_validos)]

In [16]:
data.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.05300,0.05300
1,201701,10032,20524,0,1,0.13628,0.13628
2,201701,10217,20524,0,1,0.03028,0.03028
3,201701,10125,20524,0,1,0.02271,0.02271
4,201701,10012,20524,0,11,1.54452,1.54452


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2293481 entries, 0 to 2945817
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   periodo                int64  
 1   customer_id            int64  
 2   product_id             int64  
 3   plan_precios_cuidados  int64  
 4   cust_request_qty       int64  
 5   cust_request_tn        float64
 6   tn                     float64
dtypes: float64(2), int64(5)
memory usage: 140.0 MB


In [18]:
# Transformar el campo periodo en formato fecha para aplicar al autoarima, y orfenarlos por fecha

In [19]:
# Convertir el campo 'periodo' a datetime
#data.loc[:, 'periodo'] = pd.to_datetime(data['periodo'], format='%Y%m')

In [21]:
#data['periodo'] = pd.to_datetime(data['periodo'], format='%Y%m').astype('datetime64[ns]')
data.loc[:, 'periodo'] = pd.to_datetime(data['periodo'], format='%Y%m').astype('datetime64[ns]')

In [22]:
# Filtrar datos para solo tener los necesarios
filtered_data = data[data['periodo'].dt.month.isin([1, 2, 11, 12]) & 
                     data['periodo'].dt.year.isin([2017, 2018, 2019])]

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2293481 entries, 0 to 2945817
Data columns (total 7 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   periodo                datetime64[ns]
 1   customer_id            int64         
 2   product_id             int64         
 3   plan_precios_cuidados  int64         
 4   cust_request_qty       int64         
 5   cust_request_tn        float64       
 6   tn                     float64       
dtypes: datetime64[ns](1), float64(2), int64(4)
memory usage: 140.0 MB


In [9]:
# Filtrar los datos para obtener solo los necesarios
data = data[['periodo', 'product_id', 'tn']]

# Asegurarse de que los datos están ordenados por 'periodo'
data = data.sort_values('periodo')

# Inicializar un DataFrame para almacenar las predicciones
predictions_df = pd.DataFrame(columns=['product_id', 'tn'])

# Obtener la lista de productos únicos
productos = data['product_id'].unique()

# Loop through each product
for producto in productos:
    # Filtrar los datos para el producto actual
    product_data = data[data['product_id'] == producto]
    
    # Dividir los datos en entrenamiento y prueba
    train, test = model_selection.train_test_split(product_data['tn'], train_size=0.8)
    
    # Ajustar el modelo auto_arima
    model = pm.auto_arima(train, seasonal=False, stepwise=True, trace=False)
    
    # Predecir el mes de febrero de 2020
    february_2020_pred = model.predict(n_periods=1)[0]
    
    # Agregar la predicción al DataFrame principal
    predictions_df = predictions_df.append({'product_id': producto, 'tn': february_2020_pred}, ignore_index=True)

# Guardar las predicciones en un archivo CSV
predictions_df.to_csv('february_2020_predictions.csv', index=False)

print("Predicciones para febrero de 2020 guardadas en 'february_2020_predictions.csv'")

C:\Users\hbasilio\AppData\Local\Temp\ipykernel_2664\4281199149.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<DatetimeArray>
['2017-01-01 00:00:00', '2017-01-01 00:00:00', '2017-01-01 00:00:00',
 '2017-01-01 00:00:00', '2017-01-01 00:00:00', '2017-01-01 00:00:00',
 '2017-01-01 00:00:00', '2017-01-01 00:00:00', '2017-01-01 00:00:00',
 '2017-01-01 00:00:00',
 ...
 '2019-12-01 00:00:00', '2019-12-01 00:00:00', '2019-12-01 00:00:00',
 '2019-12-01 00:00:00', '2019-12-01 00:00:00', '2019-12-01 00:00:00',
 '2019-12-01 00:00:00', '2019-12-01 00:00:00', '2019-12-01 00:00:00',
 '2019-12-01 00:00:00']
Length: 2293481, dtype: datetime64[ns]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[:, 'periodo'] = pd.to_datetime(data['periodo'], format='%Y%m')
C:\Users\hbasilio\Documents\PERSONAL\Maestria\Materias\LABO_3\ambiente-virtual\Lib\site-packages\statsmodels\tsa\base\tsa_m